## This file contains all the templates 
Each cell contains one template will executable code. Everytime train or merge the neural network, just pick one of the templates and change the running id. In this way, the repo will be expanded too fast with many similar files

### Instructions
1. pick one of cells in the `training section` and copy all the content
2. past the content in a new python file, e.g. `n1.py`. and change the id to id1
3. past the content in another new python file, e.g. `n2.py` and change the id to id2

4. pick one of cells in the `merging section` and copy all the content 
5. past the content in a new python file and run with the previous two ids

## Training section

In [ ]:
"""This cell is train_cifar0.py
This file is using CIFAR100, 
Resnet18 or shufflenet, which can be changed in the load model section
"""
#%% load dependency
import os
import numpy as np
from tqdm import tqdm
from utils import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.io import imread
from datetime import datetime
print('starting date time ', datetime.now())

#%% prepare data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR100(root='./data', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=128, shuffle=True,
    num_workers=4, pin_memory=True)

validation_loader = torch.utils.data.DataLoader(
    datasets.CIFAR100(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=128, shuffle=False,
    num_workers=4, pin_memory=True)

#%% load model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False) # original 
# model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
# model.conv1 = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model = model.cuda()

#%% train_net
id = 'CIFAR0' # for diff. runs
best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

opt = {'epochs':200}
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

loss_func = nn.CrossEntropyLoss()

for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_func(predictions, gt_label)
        loss.backward()
        optimizer.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): # don't save parameter gradients/changes since this is not for model training
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), f'teachers/teacher{id}.pt')
    scheduler.step()

#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.01, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())

In [ ]:
"""This cell is train Lung Cancer Data (LCD) 2d with individual sample
This file is using LCD and treat each slice as an independ data sample 
Resnet18 or shufflenet, which can be changed in the load model section
"""
#%% load dependency
import os
import numpy as np
from tqdm import tqdm
from utils import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.io import imread
from datetime import datetime
print('starting date time ', datetime.now())

#%% prepare data
neg, pos = torch.load('./data/LCD/neg_pos.pt') # check prep_data.py for more info
neg_all, pos_all = neg.reshape(-1,1,64,64), pos.reshape(-1,1,64,64)
if True:  # if False means split by objects
    idx = torch.randperm(pos_all.shape[0])
    neg_all = neg_all[idx]
    pos_all = pos_all[idx]  #36992, 64, 64]
num_train = 29600
num_val = 3696
num_test = 3696
split1 = num_val+num_train
split2 = num_val+num_train + num_test

train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
train_batchsize = 128
eval_batchsize = 64
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
validation_loader = DataLoader(val_dataset, eval_batchsize)
test_loader = DataLoader(test_dataset, eval_batchsize)

#%% load model
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
model.conv1 = nn.Conv2d(1, 24, kernel_size=3, stride=1, padding=1, bias=False)
model = model.cuda()

#%% train_net
id = 'LCD0' # for diff. runs
best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

opt = {'epochs':100}
optimizer = torch.optim.RAdam(model.parameters(),
                lr= 0.001,
                betas=(0.9, 0.999), 
                eps=1e-8,
                weight_decay=0)
loss_func = nn.CrossEntropyLoss()

for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    # for batch_index, (inputs, gt_label) in tqdm(enumerate(train_loader), total=len(train_dataset)//train_batchsize):
    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_func(predictions, gt_label)
        loss.backward()
        optimizer.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): # don't save parameter gradients/changes since this is not for model training
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), f'teachers/teacher{id}.pt')


#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.5, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())

In [ ]:
"""This cell is train Duke Breast Cancer (DBC) 2d with individual sample
This file is using DBC and treat each slice as an independ data sample 
Resnet18 or shufflenet, which can be changed in the load model section
"""
#%% load dependency
import os
import numpy as np
from tqdm import tqdm
from utils import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.io import imread
from datetime import datetime
print('starting date time ', datetime.now())

#%% prepare data
neg_all = torch.load('./data/DBC/neg_all.pt') # check prep_data.py for more info
pos_all = torch.load('./data/DBC/pos_all.pt')
if True:  # if False means split by objects
    idx = torch.randperm(pos_all.shape[0])
    neg_all = neg_all[idx]
    pos_all = pos_all[idx]
num_train = 2600+2200
num_val = 2200//2
num_test = 2200//2
split1 = num_val+num_train
split2 = num_val+num_train + num_test

train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
train_batchsize = 64
eval_batchsize = 32
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
validation_loader = DataLoader(val_dataset, eval_batchsize)
test_loader = DataLoader(test_dataset, eval_batchsize)

#%% load model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
# model.conv1 = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model = model.cuda()

#%% train_net
id = 'DBC0' # for diff. runs
best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

opt = {'epochs':100}
optimizer = torch.optim.RAdam(model.parameters(),
                lr= 0.001,
                betas=(0.9, 0.999), 
                eps=1e-8,
                weight_decay=0)
loss_func = nn.CrossEntropyLoss()

for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    # for batch_index, (inputs, gt_label) in tqdm(enumerate(train_loader), total=len(train_dataset)//train_batchsize):
    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_func(predictions, gt_label)
        loss.backward()
        optimizer.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): # don't save parameter gradients/changes since this is not for model training
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), f'teachers/teacher{id}.pt')


#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.5, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())

In [ ]:
"""This cell is train LCD 3d
This file is using LCD and treat each slice as an independ data sample 
Using a ResNet18-3D
"""
#%% load dependency
import os
import numpy as np
from tqdm import tqdm
from utils import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.io import imread
from datetime import datetime
print('starting date time ', datetime.now())

#%% prepare data
neg, pos = torch.load('./data/LCD/neg_pos.pt') # neg and pos each has [1156,32,64,64]
# neg = torch.cat((neg[:,:8],neg[:,8:16],neg[:,16:24], neg[:,24:]), dim=0)
# pos = torch.cat((pos[:,:8],pos[:,8:16],pos[:,16:24], pos[:,24:]), dim=0) # shape [1156*4, 8, 64, 64]
if True:  # if False means split by objects
    idx = torch.randperm(pos.shape[0])
    neg_all = neg[idx][:,None]
    pos_all = pos[idx][:,None]
num_train = 900
num_val = 156
num_test = 100
split1 = num_val+num_train
split2 = num_val+num_train + num_test

train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
train_batchsize = 128
eval_batchsize = 64
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
validation_loader = DataLoader(val_dataset, eval_batchsize)
test_loader = DataLoader(test_dataset, eval_batchsize)

#%% train_net
from modules import *
model = Resnet3D().cuda()

id = 'res3d1' # for diff. runs
fig_loc = './data/results/figures/'
mod_loc = './data/results/models/'
if not(os.path.isdir(fig_loc + f'/{id}/')): 
    print('made a new folder')
    os.makedirs(fig_loc + f'{id}/')
    os.makedirs(mod_loc + f'{id}/')
fig_loc = fig_loc + f'{id}/'
mod_loc = mod_loc + f'{id}/'

best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

opt = {'epochs':100}
optimizer = torch.optim.RAdam(model.parameters(),
                lr= 0.001,
                betas=(0.9, 0.999), 
                eps=1e-8,
                weight_decay=0)
loss_func = nn.CrossEntropyLoss()

for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    # for batch_index, (inputs, gt_label) in tqdm(enumerate(train_loader), total=len(train_dataset)//train_batchsize):
    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_func(predictions, gt_label)
        loss.backward()
        optimizer.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): # don't save parameter gradients/changes since this is not for model training
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), mod_loc+f'model_{id}.pt')

        epochs_list = list(range(epoch+1))
        plt.figure()
        plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
        plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
        plt.xlabel('epoch')
        plt.ylabel('prediction accuracy')
        plt.ylim(0.5, 1)
        plt.title('Classifier training evolution:\nprediction accuracy over time')
        plt.legend()
        plt.savefig(fig_loc+f'train_val{id}_{epoch}.png')
        plt.show()


#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.5, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(fig_loc+f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())

In [ ]:
"""This cell is train LCD 3d with costomized modified ResNet3D 
This file is using LCD and treat each slice as an independ data sample 
"""
#%% load dependency
import os
import numpy as np
from tqdm import tqdm
from utils import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.io import imread
from datetime import datetime
print('starting date time ', datetime.now())


#%% load the model
from modules import *
class Resnet3D_m(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)
        #(maxpool): Cifar does not have it.
        self.layer1 = nn.Sequential(
            BasicBlock3D(16, 16),
            BasicBlock3D(16, 16),
            )
        self.layer2 = nn.Sequential(
            BasicBlock3D(16, 16, stride=2, downsample=Downsample3D(16, 16)),
            BasicBlock3D(16, 16),
            )
        self.layer3 = nn.Sequential(
            BasicBlock3D(16, 32, stride=2, downsample=Downsample3D(16, 32)),
            BasicBlock3D(32, 32),
            )
        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(32, 2)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

#%% prepare data
neg, pos = torch.load('./data/LCD/neg_pos.pt') # neg and pos each has [1156,32,64,64]

for iseed in range(1,5):
    torch.manual_seed(iseed)
    if True:  # if False means split by objects
        idx = torch.randperm(pos.shape[0])
        neg_all = neg[idx][:,None]
        pos_all = pos[idx][:,None]
    num_train = 900
    num_val = 128
    num_test = 128
    split1 = num_val+num_train
    split2 = num_val+num_train + num_test

    train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                    torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
    val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                    torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
    test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                    torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
    train_batchsize = 64
    eval_batchsize = 64
    train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
    validation_loader = DataLoader(val_dataset, eval_batchsize)
    test_loader = DataLoader(test_dataset, eval_batchsize)

    #%% train_net
    model = Resnet3D_m().cuda()
    id = f'res3d_m{iseed}' # for diff. runs
    fig_loc = './data/results/figures/'
    mod_loc = './data/results/models/'
    if not(os.path.isdir(fig_loc + f'/{id}/')): 
        print('made a new folder')
        os.makedirs(fig_loc + f'{id}/')
        os.makedirs(mod_loc + f'{id}/')
    fig_loc = fig_loc + f'{id}/'
    mod_loc = mod_loc + f'{id}/'

    best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
    train_accs = []
    val_accs = []

    opt = {'epochs':150}
    optimizer = torch.optim.RAdam(model.parameters(),
                    lr= 0.001,
                    betas=(0.9, 0.999), 
                    eps=1e-8,
                    weight_decay=0)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(opt['epochs']):
        model.train()

        # print training info
        print("### Epoch {}:".format(epoch))
        total_train_examples = 0
        num_correct_train = 0

        # for batch_index, (inputs, gt_label) in tqdm(enumerate(train_loader), total=len(train_dataset)//train_batchsize):
        for batch_index, (inputs, gt_label) in enumerate(train_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = loss_func(predictions, gt_label)
            loss.backward()
            optimizer.step()

            _, predicted_class = predictions.max(1)
            total_train_examples += predicted_class.size(0)
            num_correct_train += predicted_class.eq(gt_label).sum().item()

        # get results
        train_acc = num_correct_train / total_train_examples
        print("Training accuracy: {}".format(train_acc))
        train_accs.append(train_acc)

        # evaluation
        total_val_examples = 0
        num_correct_val = 0
        model.eval()
        with torch.no_grad(): # don't save parameter gradients/changes since this is not for model training
            for batch_index, (inputs, gt_label) in enumerate(validation_loader):
                inputs = inputs.cuda()
                gt_label = gt_label.cuda()
                predictions = model(inputs)

                _, predicted_class = predictions.max(1)
                total_val_examples += predicted_class.size(0)
                num_correct_val += predicted_class.eq(gt_label).sum().item()

            # get validation results
            val_acc = num_correct_val / total_val_examples
            print("Validation accuracy: {}".format(val_acc))
            val_accs.append(val_acc)

        # Finally, save model if the validation accuracy is the best so far
        if val_acc > best_validation_accuracy:
            best_validation_accuracy = val_acc
            print("Validation accuracy improved; saving model.")
            torch.save(model.state_dict(), mod_loc+f'model_{id}.pt')

            epochs_list = list(range(epoch+1))
            plt.figure()
            plt.plot(epochs_list, train_accs, '-o', label='training set accuracy')
            plt.plot(epochs_list, val_accs, '--x', label='validation set accuracy')
            plt.xlabel('epoch')
            plt.ylabel('prediction accuracy')
            plt.ylim(0.5, 1)
            plt.title('Classifier training evolution:\nprediction accuracy over time')
            plt.legend()
            plt.savefig(fig_loc+f'train_val{id}_{epoch}.png')
            plt.show()

    #%% test
    model = Resnet3D_m().cuda()
    model.load_state_dict(torch.load(mod_loc+f'model_{id}.pt'))
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): 
        for batch_index, (inputs, gt_label) in enumerate(test_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Test accuracy: {}".format(val_acc))

    #%% plot train and val results
    epochs_list = list(range(opt['epochs']))
    plt.figure()
    plt.plot(epochs_list, train_accs, '-o', label='training set accuracy')
    plt.plot(epochs_list, val_accs, '--x', label='validation set accuracy')
    plt.xlabel('epoch')
    plt.ylabel('prediction accuracy')
    plt.ylim(0.5, 1)
    plt.title('Classifier training evolution:\nprediction accuracy over time')
    plt.legend()
    plt.savefig(fig_loc+f'train_val{id}.png')
    plt.show()

print('done')
print('End date time ', datetime.now())

## Merging section

In [ ]:
"""Merge the LCD 2d Resnet18
"""

#%%
from modules import *
from utils import *
from torch.utils.data import Dataset, DataLoader

#%% prepare data
neg, pos = torch.load('./data/LCD/neg_pos.pt') # check prep_data.py for more info
neg_all, pos_all = neg.reshape(-1,1,64,64), pos.reshape(-1,1,64,64)
if True:  # if False means split by objects
    idx = torch.randperm(pos_all.shape[0])
    neg_all = neg_all[idx]
    pos_all = pos_all[idx]  #36992, 64, 64]
num_train = 29600
num_val = 3696
num_test = 3696
split1 = num_val+num_train
split2 = num_val+num_train + num_test

train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
train_batchsize = 128
eval_batchsize = 64
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
validation_loader = DataLoader(val_dataset, eval_batchsize)
test_loader = DataLoader(test_dataset, eval_batchsize)

#%%
id0, id1 = 'LCD2', 'LCD3'

teacher0 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
teacher0.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False) # original
teacher0.load_state_dict(torch.load(f'teachers/teacher{id0}.pt'))

teacher1 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
teacher1.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False) # original
teacher1.load_state_dict(torch.load(f'teachers/teacher{id1}.pt'))

model = Judge(teacher0, teacher1).cuda()

#%% train_net
id = 'LCD_merge'
best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

optimizer1 = torch.optim.RAdam(model.parameters(),
                lr= 0.001,
                betas=(0.9, 0.999), 
                eps=1e-8,
                weight_decay=0)
scheduler1 = torch.optim.lr_scheduler.MultiStepLR(optimizer1, milestones=[50, 70], gamma=0.1)
optimizer2 = torch.optim.SGD(model.gate_parameters(), lr=0.2, momentum=0.9)
scheduler2 = torch.optim.lr_scheduler.MultiStepLR(optimizer2, milestones=[], gamma=1)
opt = {'epochs':100}
loss_func = nn.CrossEntropyLoss()
lamb = 0.05
for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    lamb += 0.05 * math.sqrt(lamb)
    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer1.zero_grad()
        optimizer2.zero_grad()

        predictions = model(inputs)
        main_loss = loss_func(predictions, gt_label)
        l0_loss = lamb * model.l0_loss()
        loss = main_loss + l0_loss
        loss.backward()
        optimizer1.step()
        optimizer2.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): 
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation resultsh
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), f'teachers/teacher{id}.pt')
    scheduler1.step()
    scheduler2.step()

#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.01, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())

In [ ]:
"""Merge the DBC 2d Resnet18
"""
#%%
from modules import *
from utils import *
from torch.utils.data import Dataset, DataLoader


#%% prepare data
neg_all = torch.load('./data/DBC/neg_all.pt') # check prep_data.py for more info
pos_all = torch.load('./data/DBC/pos_all.pt')
if True:  # if False means split by objects
    idx = torch.randperm(pos_all.shape[0])
    neg_all = neg_all[idx]
    pos_all = pos_all[idx]
num_train = 2600+2200
num_val = 2200//2
num_test = 2200//2
split1 = num_val+num_train
split2 = num_val+num_train + num_test

train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
train_batchsize = 64
eval_batchsize = 32
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
validation_loader = DataLoader(val_dataset, eval_batchsize)
test_loader = DataLoader(test_dataset, eval_batchsize)

#%%
id0, id1 = 'DBC0', 'DBC1'
teacher0 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
teacher0.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# teacher0.load_state_dict(torch.load(f'teachers/teacher{id0}.pt'))

teacher1 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
teacher1.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
teacher1.load_state_dict(torch.load(f'teachers/teacher{id1}.pt'))

model = Judge(teacher0, teacher1).cuda()

#%% train_net
id = 'DBC_merge_goodnbad'
best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

optimizer1 = torch.optim.RAdam(model.parameters(),
                lr= 0.001,
                betas=(0.9, 0.999), 
                eps=1e-8,
                weight_decay=0)
scheduler1 = torch.optim.lr_scheduler.MultiStepLR(optimizer1, milestones=[50, 70], gamma=0.1)
optimizer2 = torch.optim.SGD(model.gate_parameters(), lr=0.2, momentum=0.9)
scheduler2 = torch.optim.lr_scheduler.MultiStepLR(optimizer2, milestones=[], gamma=1)
opt = {'epochs':100}
loss_func = nn.CrossEntropyLoss()
lamb = 0.05
for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    lamb += 0.05 * math.sqrt(lamb)
    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer1.zero_grad()
        optimizer2.zero_grad()

        predictions = model(inputs)
        main_loss = loss_func(predictions, gt_label)
        l0_loss = lamb * model.l0_loss()
        loss = main_loss + l0_loss
        loss.backward()
        optimizer1.step()
        optimizer2.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): 
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), f'teachers/teacher{id}.pt')
    scheduler1.step()
    scheduler2.step()

#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.01, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())

In [ ]:
"""Merge the LCD 3d Resnet18
"""
#%% load dependency
import os
import numpy as np
from tqdm import tqdm
from utils import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.io import imread
from datetime import datetime
print('starting date time ', datetime.now())

#%% prepare data
neg, pos = torch.load('./data/LCD/neg_pos.pt') # neg and pos each has [1156,32,64,64]
neg = torch.cat((neg[:,:8],neg[:,8:16],neg[:,16:24], neg[:,24:]), dim=0)
pos = torch.cat((pos[:,:8],pos[:,8:16],pos[:,16:24], pos[:,24:]), dim=0) # shape [1156*4, 8, 64, 64]
if True:  # if False means split by objects
    idx = torch.randperm(pos.shape[0])
    neg_all = neg[idx][:,None]
    pos_all = pos[idx][:,None]
num_train = 3600
num_val = 512
num_test = 512
split1 = num_val+num_train
split2 = num_val+num_train + num_test

train_dataset = Data.TensorDataset(torch.cat((pos_all[:num_train], neg_all[:num_train]), dim=0), \
                torch.cat((torch.ones(num_train, dtype=int), torch.zeros(num_train,  dtype=int)), dim=0))
val_dataset = Data.TensorDataset(torch.cat((pos_all[num_train:split1],neg_all[num_train:split1]), dim=0), \
                torch.cat((torch.ones(num_val, dtype=int), torch.zeros(num_val, dtype=int)), dim=0))
test_dataset = Data.TensorDataset(torch.cat((pos_all[split1:split2], neg_all[split1:split2]), dim=0), \
                torch.cat((torch.ones(num_test, dtype=int), torch.zeros(num_test, dtype=int)), dim=0))
train_batchsize = 128
eval_batchsize = 64
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)                                      
validation_loader = DataLoader(val_dataset, eval_batchsize)
test_loader = DataLoader(test_dataset, eval_batchsize)

#%%
from modules import Resnet3D, Judge3D
id0, id1 = 'DBC0', 'res3d1'
client0 = Resnet3D()
# client0.load_state_dict(torch.load(f'teachers/teacher{id0}.pt'))

client1 = Resnet3D()
client1.load_state_dict(torch.load(f'./data/results/models/{id1}/model_{id1}.pt'))

model = Judge3D(client0, client1).cuda()

#%% train_net
id = 'res3d1_and_rand' # for diff. runs
fig_loc = './data/results/merege/figures/'
mod_loc = './data/results/merge/models/'
if not(os.path.isdir(fig_loc + f'/{id}/')): 
    print('made a new folder')
    os.makedirs(fig_loc + f'{id}/')
    os.makedirs(mod_loc + f'{id}/')
fig_loc = fig_loc + f'{id}/'
mod_loc = mod_loc + f'{id}/'
best_validation_accuracy = 0. # used to pick the best-performing model on the validation set
train_accs = []
val_accs = []

optimizer1 = torch.optim.RAdam(model.parameters(),
                lr= 0.001,
                betas=(0.9, 0.999), 
                eps=1e-8,
                weight_decay=0)
scheduler1 = torch.optim.lr_scheduler.MultiStepLR(optimizer1, milestones=[50, 70], gamma=0.1)
optimizer2 = torch.optim.SGD(model.gate_parameters(), lr=0.2, momentum=0.9)
scheduler2 = torch.optim.lr_scheduler.MultiStepLR(optimizer2, milestones=[], gamma=1)
opt = {'epochs':100}
loss_func = nn.CrossEntropyLoss()
lamb = 0.05
for epoch in range(opt['epochs']):
    model.train()

    # print training info
    print("### Epoch {}:".format(epoch))
    total_train_examples = 0
    num_correct_train = 0

    lamb += 0.05 * (lamb**0.5)
    for batch_index, (inputs, gt_label) in enumerate(train_loader):
        inputs = inputs.cuda()
        gt_label = gt_label.cuda()
        optimizer1.zero_grad()
        optimizer2.zero_grad()

        predictions = model(inputs)
        main_loss = loss_func(predictions, gt_label)
        l0_loss = lamb * model.l0_loss()
        loss = main_loss + l0_loss
        loss.backward()
        optimizer1.step()
        optimizer2.step()

        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)
        num_correct_train += predicted_class.eq(gt_label).sum().item()

    # get results
    train_acc = num_correct_train / total_train_examples
    print("Training accuracy: {}".format(train_acc))
    train_accs.append(train_acc)

    # evaluation
    total_val_examples = 0
    num_correct_val = 0
    model.eval()
    with torch.no_grad(): 
        for batch_index, (inputs, gt_label) in enumerate(validation_loader):
            inputs = inputs.cuda()
            gt_label = gt_label.cuda()
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(gt_label).sum().item()

        # get validation results
        val_acc = num_correct_val / total_val_examples
        print("Validation accuracy: {}".format(val_acc))
        val_accs.append(val_acc)

    # Finally, save model if the validation accuracy is the best so far
    if val_acc > best_validation_accuracy:
        best_validation_accuracy = val_acc
        print("Validation accuracy improved; saving model.")
        torch.save(model.state_dict(), f'teachers/teacher{id}.pt')

        epochs_list = list(range(epoch+1))
        plt.figure()
        plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
        plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
        plt.xlabel('epoch')
        plt.ylabel('prediction accuracy')
        plt.ylim(0.5, 1)
        plt.title('Classifier training evolution:\nprediction accuracy over time')
        plt.legend()
        plt.savefig(fig_loc+f'train_val{id}_{epoch}.png')
        plt.show()

    scheduler1.step()
    scheduler2.step()

#%% plot train and val results
epochs_list = list(range(opt['epochs']))
plt.figure()
plt.plot(epochs_list, train_accs, 'b-', label='training set accuracy')
plt.plot(epochs_list, val_accs, 'r-', label='validation set accuracy')
plt.xlabel('epoch')
plt.ylabel('prediction accuracy')
plt.ylim(0.01, 1)
plt.title('Classifier training evolution:\nprediction accuracy over time')
plt.legend()
plt.savefig(f'train_val{id}.png')
plt.show()

print('done')
print('End date time ', datetime.now())